In [ ]:
pip install -U ultralytics

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="rWOwsHdmSVhYNPUUvRtM")
project = rf.workspace("pest-detection-lofdq").project("pests-detection-gsgkn")
version = project.version(3)
dataset = version.download("yolov5")

# Check if dataset is downloaded correctly
!mkdir -p /content/datasets
!ls /content
!ls /content/datasets  # Confirm the directory where the dataset is saved


In [ ]:
# Load and display the current contents of data.yaml
import yaml

yaml_path = '/content/Pests-Detection-3/data.yaml'

# Open and print the contents of data.yaml
with open(yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)

print(data_yaml)


In [ ]:
# Update the paths if incorrect
data_yaml['train'] = '/content/Pests-Detection-3/train/images'
data_yaml['val'] = '/content/Pests-Detection-3/valid/images'

# Save the updated data.yaml file
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file)

print("Updated paths in data.yaml:", data_yaml)


In [ ]:
with open('/content/Pests-Detection-3/data.yaml', 'r') as f:
    print(f.read())


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/Pests-Detection-3/data.yaml epochs=50 imgsz=640


In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/Pests-Detection-3/data.yaml

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source=/content/Pests-Detection-3/test/images save=true

In [ ]:
!zip -r /content/runs.zip /content/runs

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'/content/runs/detect/predict/display630_3_jpg.rf.8feede90c83a2a65517bda2feae20c74.jpg')[:3]:
      display(Image(filename=image_path, height=600))
      print("\n")

In [ ]:
!pip install pyngrok
!ngrok config add-authtoken "2o6bFU2iXAJBrOUQdxmmtGo9uWG_674R6gTVXJnQNe3hjZWuN"

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Ngrok Tunnel URL:", public_url)

In [ ]:
from flask import Flask, request
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Define the endpoint to receive data
@app.route('/your_endpoint', methods=['POST'])
def receive_data():
    data = request.form
    distance = data.get("distance")
    pir = data.get("pir")
    print(f"Distance: {distance}, PIR: {pir}")
    return "Data received", 200

# Start Ngrok tunnel and Flask app
public_url = ngrok.connect(5000)
print("Ngrok Tunnel URL:", public_url)
app.run(port=5000)
